# Final Project Phase 2 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 2 of the Final Project. Answer all statements addressed below as specified in the instructions for the project, covering all necessary details. Please be clear and concise in your answers. Each response should be at most 3 sentences. Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

## Jupyter Notebook Quick Tips
Here are some quick formatting tips to get you started with Jupyter Notebooks. This is by no means exhaustive, and there are plenty of articles to highlight other things that can be done. We recommend using HTML syntax for Markdown but there is also Markdown syntax that is more streamlined and might be preferable.
<a href = "https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671">Here's an article</a> that goes into more detail. (Double-click on cell to see syntax)

# Heading 1
## Heading 2
### Heading 3
#### Heading 4
<br>
<b>BoldText</b> or <i>ItalicText</i>
<br> <br>
Math Formulas: $x^2 + y^2 = 1$
<br> <br>
Line Breaks are done using br enclosed in < >.
<br><br>
Hyperlinks are done with: <a> https://www.google.com </a> or
<a href="http://www.google.com">Google</a><br>

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file.

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.


## Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
*   Downloaded Dataset Source: https://apps.who.int/nha/database/Select/Indicators/en
*   Web Collection #1 Source: https://en.wikipedia.org/wiki/List_of_countries_by_total_health_expenditure_per_capita
*   Web Collection #2 Source: https://echarts.apache.org/examples/data/asset/data/life-expectancy-table.json



## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [ ]:
import pandas as pd
import requests
from pprint import pprint
def data_parser():
  df = pd.read_excel("/content/GHED_data.xlsx")
  df1 = df.iloc[:,0:8]
  df1 = df1.dropna()
  df1 = df1.set_index('country')
  df1['year'] = df1['year'].astype(int)
  df1 = df1[(df1['year'] == 2002) | (df1['year']== 2010) | (df1['year']== 2018)]
  df1.loc[::2,'che_pc_usd'] *= 1.28
  del df1['country code']
  del df1['region (WHO)']
  del df1['income group']
  df1 = df1.round(2)
  df1 = df1.rename(columns = {"che_gdp": 'Current Health Expenditure as % of GDP', 'che_pc_usd':'Current Health Expenditure per Capita in US dollars (Millions)','che_pc_ppp':'Current Health Expenditure per Capita in Purchasing Power Parity (PPP)'})
  df1 = df1.rename(index = {'Viet Nam': 'Vietnam'})
  df1.to_csv("cleanedGHEDdata.csv",index= True)
  pprint(df1)

# Sources Used: https://www.statology.org/pandas-select-multiple-columns/
# https://www.rba.gov.au/calculator/annualDecimal.html
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html





############ Function Call ############
data_parser()

         year  Current Health Expenditure as % of GDP  \
country                                                 
Algeria  2002                                    3.73   
Algeria  2010                                    5.12   
Algeria  2018                                    6.16   
Angola   2002                                    3.33   
Angola   2010                                    2.70   
...       ...                                     ...   
Vanuatu  2010                                    3.53   
Vanuatu  2018                                    3.35   
Vietnam  2002                                    3.60   
Vietnam  2010                                    4.70   
Vietnam  2018                                    5.03   

         Current Health Expenditure per Capita in US dollars (Millions)  \
country                                                                   
Algeria                                              85.07                
Algeria                          

## Web Collection Requirement \#1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pprint import pprint
def web_parser1():
  listy = []
  r = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_total_health_expenditure_per_capita')
  soup = BeautifulSoup(r.text, "html.parser")
  taglist = soup.find_all("tr")[38:-13]
  for row in taglist:
    info = row.find_all('td')
    infolist = []
    for info2 in info:
      infolist.append(info2.text)
    listy.append(infolist)
  df = pd.DataFrame(listy)
  df = df.dropna()

  df.columns = ['Country', '2002', '2010', '2018']
  df['Country'] = df['Country'].str.strip('*\n')
  df['2002'] = df['2002'].str.strip('*\n')
  df['2002'] = df['2002'].str.replace(',', '')
  df['2010'] = df['2010'].str.strip('*\n')
  df['2010'] = df['2010'].str.replace(',', '')
  df['2018'] = df['2018'].str.strip('*\n')
  df['2018'] = df['2018'].str.replace(',', '')
  df = df.mask(df == '')
  df = df.dropna()
  df.iloc[95, :] = ['Liberia',9, 45, 61]
  df['2002'] = df['2002'].astype('int')
  df['2010'] = df['2010'].astype('int')
  df['2018'] = df['2018'].astype('int')
  df['2002-2010 perecent change'] = round((df['2010']/df['2002']), 3)
  df['2010-2018 perecent change'] = round((df['2018']/df['2010']), 3)
  df['2026 Projected Health Expenditure Per Capita, adjusted to 2018 US Dollar Inflation'] = round(((((df['2010']/df['2002']) + (df['2018']/df['2010'])) / 2) * df['2018']), 2)
  df = df.rename(columns = {'2002': '2002 Health Expenditure Per Capita, adjusted to 2018 US Dollar Inflation', '2010' : '2010 Health Expenditure Per Capita, adjusted to 2018 US Dollar Inflation', '2018':'2018 Health Expenditure Per Capita, adjusted to 2018 US Dollar Inflation'})
  df = df.set_index('Country')
  df.to_csv("cleanedwebscrapeddata.csv",index=True)


  pprint(df)


# Sources Used: https://www.geeksforgeeks.org/pprint-data-pretty-printer-python/
# https://stackoverflow.com/questions/29461185/filtering-pandas-dataframe-using-or-statement






############ Function Call ############
web_parser1()

               2002 Health Expenditure Per Capita, adjusted to 2018 US Dollar Inflation  \
Country                                                                                   
 Afghanistan                                                  78                          
 Albania                                                     314                          
 Algeria                                                     335                          
 Andorra                                                    2196                          
 Angola                                                      119                          
...                                                          ...                          
 Uzbekistan                                                  175                          
 Vanuatu                                                      80                          
 Venezuela                                                   842                          

## Web Collection Requirement \#2

In [ ]:
import requests
import pandas as pd
from pprint import pprint
def web_parser2():
  df = pd.read_json("https://echarts.apache.org/examples/data/asset/data/life-expectancy-table.json")
  df.columns =df.iloc[0]
  df = df[1:]
  del df['Population']
  del df['Income']
  df = df.set_index('Country')
  yearle = df.groupby('Year')['Life Expectancy'].mean() #returns pd dataframe that shows increase in average life expecatncy around the world thru the years
  df.to_csv("cleanedlifeexpectancy.csv",index = True)
  pprint(df)


  # Sources Used: https://www.askpython.com/python-modules/pandas/update-the-value-of-a-row-dataframe
#  https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671








############ Function Call ############
web_parser2()

0              Life Expectancy  Year
Country                             
Australia                34.05  1800
Canada                      39  1800
China                       32  1800
Cuba                      32.2  1800
Finland              36.573126  1800
...                        ...   ...
Poland                    77.3  2015
Russia                   73.13  2015
Turkey                    76.5  2015
United Kingdom            81.4  2015
United States             79.1  2015

[1539 rows x 2 columns]


## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [ ]:
def extra_source1():
    pass


############ Function Call ############
extra_source1()

In [ ]:
# Define further extra source functions as necessary

#Inconsistencies
For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. In the web scraping data, there were a few blank cells in either of the 2002, 2010, 2018 health expenditure columns. This prevented us from creating columns such as percent change columns and predicting future health expenditure of countries. To take care of these blank cells, we decided to fill the blanks with NaN values and then remove the rows with these values using df.dropna(). We could not replace these NaN values with 0 because it would mess with the numbers of the columns we wish to create. This allowed us to create the percent change between the different years and also predict the 2026 health expenditure of different countries.


2. Another inconsistency faced in the web scraped dataset was that the country Liberia had numbers that were incorrect (numbers were abnormally inflated). We discovered this while going through the table to check for any mistakes. To change this, We got numbers from https://www.macrotrends.net/countries/LBR/liberia/healthcare-spending, and we changed the values for Liberia in the code to reflect the new numbers we found. Now, there are more accurate numbers that the code can use while creating the new columns.


3. In the large CSV downloaded dataset, there were many columns with unnecessary information and rows that contained blank cells, so we decided to replace all these blank cells with 0 and disregard the cells with 0 when analyzing the data during phase 3.


4. In the downloaded CSV dataset, the rows for the country Vietnam were different from the other data sets. Originally, the dataset displayed the country name as “Viet Nam'' which is different from how the other datasets represented the country so these rows had to be changed to “Vietnam.” Also this dataset had a bunch of code names like “che_gdh” which represented the “Current Health Expenditure for Countries GDP per capita.” We had to change these code names into labels that would make sense to the reader and to match the names of the columns of the other datasets.

5. Our web scraping dataset had columns where the numbers were adjusted to 2018 inflation, so we decided to adjust the values in the CSV downloaded dataset to 2018 inflation as well to keep the numbers consistent across all datasets.

